<a href="https://colab.research.google.com/github/mjgpinheiro/Physics_models/blob/main/MPD_Geometric_Stability_Analysis_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import eigh
from scipy.sparse import diags
from scipy.sparse.linalg import eigsh
import matplotlib.gridspec as gridspec
import warnings
warnings.filterwarnings('ignore')

# Set publication-quality plotting style
plt.rcParams.update({
    'font.size': 11,
    'font.family': 'serif',
    'font.serif': ['Times New Roman'],
    'axes.labelsize': 12,
    'axes.titlesize': 12,
    'xtick.labelsize': 10,
    'ytick.labelsize': 10,
    'legend.fontsize': 10,
    'figure.dpi': 300,
    'savefig.dpi': 300,
    'savefig.bbox': 'tight',
    'lines.linewidth': 2,
})

print("="*80)
print("CORRECTED MHD STABILITY ANALYSIS - FULL STURM-LIOUVILLE PROBLEM")
print("="*80)

#=============================================================================
# CLASS DEFINITION WITH FULL RADIAL OPERATOR
#=============================================================================

class MPDStabilityAnalyzerCorrected:
    """Analyzes MPD thruster stability using full Sturm-Liouville problem"""

    def __init__(self, R=0.1, B0=0.5, J0=5e6, k=20, rho=1e-5,
                 mu0=4*np.pi*1e-7, N=200):
        """
        Parameters:
        -----------
        R : float
            Thruster radius (m)
        B0 : float
            Axial magnetic field (T)
        J0 : float
            Current density (A/m²)
        k : float
            Axial wavenumber (m⁻¹)
        rho : float
            Plasma density (kg/m³)
        mu0 : float
            Permeability of free space
        N : int
            Number of radial grid points
        """
        self.R = R
        self.B0 = B0
        self.J0 = J0
        self.k = k
        self.rho = rho
        self.mu0 = mu0
        self.N = N

        # Derived parameters
        self.C = mu0 * J0 / 2
        self.r_grid = np.linspace(0, R, N)
        self.r_mid = R/2
        self.dr = self.r_grid[1] - self.r_grid[0]

        # Compute coefficient functions A(r), B(r), C(r) from Eq. (10-11)
        self.A = self.B0**2 / self.mu0 * np.ones_like(self.r_grid)
        self.B_coeff = (self.k**2 * self.B0**2 + 3 * self.C**2 * self.r_grid**2) / self.mu0 \
                       + 2 * self.C**2 * self.r_grid**2 / self.mu0 \
                       + (1/self.r_grid) * np.gradient(self.r_grid * self.B0**2 / self.mu0, self.dr)
        self.C_coeff = 2 * self.B0**2 / (self.mu0 * self.r_grid)

        # Handle r=0 singularity
        self.B_coeff[0] = self.B_coeff[1]
        self.C_coeff[0] = 0

        # Compute G(r)
        self.G_r = self.compute_G(self.r_grid)
        # Ensure G_mid is computed correctly for a scalar r_mid
        self.G_mid = self.compute_G(np.array([self.r_mid]))[0] # Pass as array and take first element

    def compute_G(self, r):
        """Compute geometric stability index G at radius r"""
        with np.errstate(divide='ignore', invalid='ignore'):
            G = (4 * self.k**2 * self.B0**2) / (3 * self.mu0**2 * self.J0**2 * r**2)
            # Apply np.inf only if r is an array and contains 0
            if isinstance(r, np.ndarray):
                G[r == 0] = np.inf
            elif r == 0:
                G = np.inf
        return G

    def build_fem_matrices(self):
        """
        Build finite element matrices for generalized eigenvalue problem:
        K ξ = λ M ξ

        Using linear finite elements on a uniform grid.
        """
        N = self.N
        dr = self.dr
        r = self.r_grid

        # Stiffness matrix K (includes derivative terms)
        K = np.zeros((N, N))

        # Mass matrix M
        M = np.zeros((N, N))

        # Assemble elements (linear elements)
        for i in range(N-1):
            r_i = r[i]
            r_ip1 = r[i+1]

            # Element length
            h = dr

            # Average coefficients on element
            A_avg = 0.5 * (self.A[i] + self.A[i+1])
            B_avg = 0.5 * (self.B_coeff[i] + self.B_coeff[i+1])
            C_avg = 0.5 * (self.C_coeff[i] + self.C_coeff[i+1])
            r_avg = 0.5 * (r_i + r_ip1)

            # Element stiffness matrix (local)
            # ∫ (A dξ/dr dφ/dr + B ξ φ + C ξ dφ/dr) r dr
            k_loc = np.zeros((2, 2))

            # dξ/dr * dφ/dr term
            k_loc[0, 0] += A_avg * r_avg / h
            k_loc[0, 1] += -A_avg * r_avg / h
            k_loc[1, 0] += -A_avg * r_avg / h
            k_loc[1, 1] += A_avg * r_avg / h

            # ξ * φ term (mass term) - using trapezoidal rule
            k_loc[0, 0] += B_avg * r_i * h / 3
            k_loc[0, 1] += B_avg * r_avg * h / 6
            k_loc[1, 0] += B_avg * r_avg * h / 6
            k_loc[1, 1] += B_avg * r_ip1 * h / 3

            # ξ * dφ/dr term (non-symmetric contribution)
            # This requires careful treatment; simplified here
            k_loc[0, 0] += C_avg * r_avg * 0.25
            k_loc[0, 1] += C_avg * r_avg * 0.25
            k_loc[1, 0] += -C_avg * r_avg * 0.25
            k_loc[1, 1] += -C_avg * r_avg * 0.25

            # Assemble into global matrix
            K[i, i] += k_loc[0, 0]
            K[i, i+1] += k_loc[0, 1]
            K[i+1, i] += k_loc[1, 0]
            K[i+1, i+1] += k_loc[1, 1]

            # Mass matrix for generalized eigenvalue problem
            # M = ∫ ρ r ξ φ dr
            m_loc = np.zeros((2, 2))
            m_loc[0, 0] += self.rho * r_i * h / 3
            m_loc[0, 1] += self.rho * r_avg * h / 6
            m_loc[1, 0] += self.rho * r_avg * h / 6
            m_loc[1, 1] += self.rho * r_ip1 * h / 3

            M[i, i] += m_loc[0, 0]
            M[i, i+1] += m_loc[0, 1]
            M[i+1, i] += m_loc[1, 0]
            M[i+1, i+1] += m_loc[1, 1]

        # Apply boundary conditions
        # ξ(0) = 0 (fixed at center)
        K[0, :] = 0
        K[:, 0] = 0
        K[0, 0] = 1
        M[0, :] = 0
        M[0, 0] = 1

        # dξ/dr(R) = 0 (natural boundary condition - already satisfied by FEM)

        return K, M

    def solve_eigenproblem(self):
        """Solve generalized eigenvalue problem Kξ = λMξ"""
        K, M = self.build_fem_matrices()

        # Use sparse solver for efficiency
        try:
            eigenvalues, eigenvectors = eigsh(K, k=20, M=M, which='SM')
        except:
            # Fall back to dense solver
            eigenvalues = eigh(K, M, eigvals_only=True)
            eigenvalues = eigenvalues[:20]

        return np.sort(eigenvalues)

    def analyze_stability_scan(self, B0_scale=None):
        """Analyze stability as function of B0 scaling"""
        if B0_scale is None:
            B0_scale = np.linspace(0.1, 2.0, 30)

        min_evals = []
        G_values = []

        for scale in B0_scale:
            # Create temporary analyzer with scaled B0
            temp = MPDStabilityAnalyzerCorrected(
                R=self.R,
                B0=self.B0 * scale,
                J0=self.J0,
                k=self.k,
                rho=self.rho,
                mu0=self.mu0,
                N=self.N
            )

            # Compute eigenvalues
            evals = temp.solve_eigenproblem()
            min_evals.append(evals[0])
            G_values.append(temp.G_mid)

        return np.array(B0_scale), np.array(G_values), np.array(min_evals)


# Initialize corrected analyzer
print("\nInitializing MPD thruster configuration with FULL RADIAL OPERATOR:")
analyzer = MPDStabilityAnalyzerCorrected()
print(f"  Radius R = {analyzer.R} m")
print(f"  Axial field B₀ = {analyzer.B0} T")
print(f"  Current density J₀ = {analyzer.J0:.1e} A/m²")
print(f"  Axial wavenumber k = {analyzer.k} m⁻¹")
print(f"  Plasma density ρ = {analyzer.rho} kg/m³")
print(f"  Geometric index at r=R/2: G = {analyzer.G_mid:.1f}")

# Compute eigenvalues with full operator
print("\nSolving full Sturm-Liouville problem...")
eigenvalues_full = analyzer.solve_eigenproblem()
print(f"  Minimum eigenvalue: λ_min = {eigenvalues_full[0]:.4e}")
print(f"  First 5 eigenvalues: {eigenvalues_full[:5]}")

# Perform stability scan
print("\nPerforming stability scan over B₀...")
B0_scales, G_values, min_evals = analyzer.analyze_stability_scan()

# Find critical point where λ_min ≈ 0
critical_idx = np.argmin(np.abs(min_evals))
critical_scale = B0_scales[critical_idx]
critical_G = G_values[critical_idx]
critical_B0 = critical_scale * analyzer.B0

print(f"\nCritical parameters:")
print(f"  B₀_crit = {critical_B0:.3f} T")
print(f"  G_crit = {critical_G:.2f}")
print(f"  Scale factor = {critical_scale:.2f}")

#=============================================================================
# FIGURE 1: Equilibrium Configuration (same as before)
#=============================================================================

print("\nGenerating Figure 1: Equilibrium Configuration...")
fig1 = plt.figure(figsize=(10, 8))
gs1 = gridspec.GridSpec(2, 2, hspace=0.3, wspace=0.3)

# (a) Effective potential (now including full expression)
ax1a = plt.subplot(gs1[0, 0])
V_full = analyzer.B_coeff  # This is the full potential including all terms
ax1a.plot(analyzer.r_grid*1000, V_full/1e6, 'b-', linewidth=2.5)
ax1a.set_xlabel('Radius (mm)')
ax1a.set_ylabel('V$_{\\mathrm{eff}}$ (MJ/m³)')
ax1a.set_title('(a) Effective Potential (Full)')
ax1a.grid(True, alpha=0.3)

# (b) Magnetic field configuration
ax1b = plt.subplot(gs1[0, 1])
B_theta = analyzer.C * analyzer.r_grid
ax1b.plot(analyzer.r_grid*1000, B_theta, 'r-', linewidth=2.5, label='B$_\\theta$(r)')
ax1b.axhline(y=analyzer.B0, color='b', linestyle='--', linewidth=2, label='B$_z$ = const')
ax1b.set_xlabel('Radius (mm)')
ax1b.set_ylabel('Magnetic Field (T)')
ax1b.set_title('(b) Magnetic Field Configuration')
ax1b.legend(loc='upper left')
ax1b.grid(True, alpha=0.3)

# (c) Pressure profile
ax1c = plt.subplot(gs1[1, 0])
p_profile = (analyzer.r_grid*1000)**2 / 100
ax1c.plot(analyzer.r_grid*1000, p_profile, 'g-', linewidth=2.5)
ax1c.set_xlabel('Radius (mm)')
ax1c.set_ylabel('p (normalized)')
ax1c.set_title('(c) Equilibrium Pressure')
ax1c.grid(True, alpha=0.3)

# (d) Coefficient functions A(r), B(r), C(r)
ax1d = plt.subplot(gs1[1, 1])
ax1d.plot(analyzer.r_grid*1000, analyzer.A, 'r-', label='A(r)')
ax1d.plot(analyzer.r_grid*1000, analyzer.B_coeff/1e6, 'b-', label='B(r) (MJ/m³)')
ax1d.plot(analyzer.r_grid*1000, analyzer.C_coeff, 'g-', label='C(r)')
ax1d.set_xlabel('Radius (mm)')
ax1d.set_ylabel('Coefficient Value')
ax1d.set_title('(d) Operator Coefficients')
ax1d.legend(fontsize=8)
ax1d.grid(True, alpha=0.3)

plt.suptitle('Figure 1: Equilibrium Configuration and Operator Coefficients', fontsize=14, y=1.02)
plt.tight_layout()
plt.savefig('Fig1_equilibrium_potential_corrected.png', dpi=300, bbox_inches='tight')
plt.close(fig1)
print("  ✓ Fig1_equilibrium_potential_corrected.png saved")

#=============================================================================
# FIGURE 2: Geometric Stability Index G(r)
#=============================================================================

print("Generating Figure 2: Geometric Stability Index...")
fig2 = plt.figure(figsize=(8, 6))
ax2 = plt.subplot(111)

ax2.semilogy(analyzer.r_grid*1000, analyzer.G_r, 'g-', linewidth=2.5, label='G(r)')
ax2.axhline(y=1, color='k', linestyle=':', linewidth=2, label='G = 1')
ax2.axhline(y=critical_G, color='r', linestyle='--', linewidth=2,
            label=f'G$_{{\\mathrm{{crit}}}}$ = {critical_G:.1f}')
ax2.axhline(y=analyzer.G_mid, color='b', linestyle='-.', linewidth=2,
            label=f'G(r=R/2) = {analyzer.G_mid:.1f}')

ax2.fill_between(analyzer.r_grid*1000, 1, analyzer.G_r, where=(analyzer.G_r>1),
                 color='green', alpha=0.2, label='Stable (G > 1)')
ax2.fill_between(analyzer.r_grid*1000, 0, analyzer.G_r, where=(analyzer.G_r<1),
                 color='red', alpha=0.2, label='Unstable (G < 1)')

ax2.set_xlabel('Radius (mm)')
ax2.set_ylabel('Geometric Stability Index G(r)')
ax2.set_title('Figure 2: Radial Profile of Geometric Stability Index')
ax2.legend(loc='upper right')
ax2.grid(True, alpha=0.3)
ax2.set_xlim([0, 100])
ax2.set_ylim([1e-1, 1e4])

plt.tight_layout()
plt.savefig('Fig2_geometric_index_corrected.png', dpi=300, bbox_inches='tight')
plt.close(fig2)
print("  ✓ Fig2_geometric_index_corrected.png saved")

#=============================================================================
# FIGURE 3: Hessian Eigenvalue Spectrum (Full Operator)
#=============================================================================

print("Generating Figure 3: Hessian Eigenvalue Spectrum (Full Operator)...")
fig3 = plt.figure(figsize=(8, 6))
ax3 = plt.subplot(111)

modes = np.arange(1, len(eigenvalues_full)+1)
ax3.semilogy(modes, eigenvalues_full, 'bo-', markersize=6, linewidth=2,
            markerfacecolor='white', markeredgewidth=1.5)

ax3.axhline(y=0, color='k', linestyle='-', alpha=0.5, linewidth=1)
ax3.annotate(f'λ_min = {eigenvalues_full[0]:.2e}',
             xy=(5, eigenvalues_full[0]), xytext=(10, eigenvalues_full[0]*2),
             arrowprops=dict(arrowstyle='->', color='red', linewidth=1.5),
             fontsize=11, bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

ax3.set_xlabel('Mode Index')
ax3.set_ylabel('Eigenvalue λ')
ax3.set_title('Figure 3: Hessian Eigenvalue Spectrum (Full Sturm-Liouville)')
ax3.grid(True, alpha=0.3, which='both')
ax3.set_xlim([0, 21])

plt.tight_layout()
plt.savefig('Fig3_eigenvalue_spectrum_corrected.png', dpi=300, bbox_inches='tight')
plt.close(fig3)
print("  ✓ Fig3_eigenvalue_spectrum_corrected.png saved")

#=============================================================================
# FIGURE 4: Stability Threshold Analysis
#=============================================================================

print("Generating Figure 4: Stability Threshold Analysis...")
fig4 = plt.figure(figsize=(8, 6))
ax4 = plt.subplot(111)

ax4.plot(B0_scales, min_evals/1e6, 'b-', linewidth=2.5, label='λ_min')
ax4.axhline(y=0, color='k', linestyle='-', alpha=0.5, linewidth=1)
ax4.axvline(x=critical_scale, color='r', linestyle='--', linewidth=2,
            label=f'B₀/B₀₀ = {critical_scale:.2f}')

ax4.fill_between(B0_scales, 0, min_evals/1e6, where=(min_evals>0),
                 color='green', alpha=0.3, label='Stable')
ax4.fill_between(B0_scales, min_evals/1e6, 0, where=(min_evals<0),
                 color='red', alpha=0.3, label='Unstable')

ax4.plot(critical_scale, 0, 'ro', markersize=8, markerfacecolor='red')

ax4.set_xlabel('B₀ / B₀₀')
ax4.set_ylabel('λ_min (×10⁶)')
ax4.set_title('Figure 4: Stability Threshold as Function of Axial Field')
ax4.legend(loc='upper left')
ax4.grid(True, alpha=0.3)
ax4.set_xlim([0.1, 2.0])
ax4.set_ylim([-50, 150])

plt.tight_layout()
plt.savefig('Fig4_stability_threshold_corrected.png', dpi=300, bbox_inches='tight')
plt.close(fig4)
print("  ✓ Fig4_stability_threshold_corrected.png saved")

#=============================================================================
# FIGURE 5: G vs λ_min Correlation
#=============================================================================

print("Generating Figure 5: G vs λ_min Correlation...")
fig5 = plt.figure(figsize=(8, 6))
ax5 = plt.subplot(111)

ax5.plot(G_values, min_evals/1e6, 'bo-', markersize=6, linewidth=2,
         markerfacecolor='white', markeredgewidth=1.5)
ax5.axhline(y=0, color='k', linestyle='-', alpha=0.5, linewidth=1)
ax5.axvline(x=critical_G, color='r', linestyle='--', linewidth=2,
            label=f'G_crit = {critical_G:.1f}')

ax5.fill_between(G_values, 0, min_evals/1e6, where=(min_evals>0),
                 color='green', alpha=0.3)
ax5.fill_between(G_values, min_evals/1e6, 0, where=(min_evals<0),
                 color='red', alpha=0.3)

ax5.plot(critical_G, 0, 'ro', markersize=8, markerfacecolor='red')

ax5.set_xlabel('Geometric Stability Index G')
ax5.set_ylabel('λ_min (×10⁶)')
ax5.set_title('Figure 5: Correlation Between G and Minimum Eigenvalue')
ax5.legend(loc='upper left')
ax5.grid(True, alpha=0.3)

# Add inset showing near-threshold behavior
ax5_inset = fig5.add_axes([0.55, 0.6, 0.25, 0.25])
near_idx = np.where((G_values > 0) & (G_values < 200))[0]
ax5_inset.plot(G_values[near_idx], min_evals[near_idx]/1e6, 'b-', linewidth=2)
ax5_inset.axhline(y=0, color='k', linestyle='-', alpha=0.5)
ax5_inset.axvline(x=critical_G, color='r', linestyle='--')
ax5_inset.plot(critical_G, 0, 'ro', markersize=6)
ax5_inset.set_xlabel('G')
ax5_inset.set_ylabel('λ_min (×10⁶)')
ax5_inset.set_title('Near Threshold')
ax5_inset.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('Fig5_G_lambda_correlation_corrected.png', dpi=300, bbox_inches='tight')
plt.close(fig5)
print("  ✓ Fig5_G_lambda_correlation_corrected.png saved")

#=============================================================================
# FIGURE 6: Design Space - G Contours
#=============================================================================

print("Generating Figure 6: Design Space - G Contours...")

B0_range = np.logspace(-1, 1, 50)
J0_range = np.logspace(5, 7, 50)
B0_mesh, J0_mesh = np.meshgrid(B0_range, J0_range)

r_mid = analyzer.R / 2
k = analyzer.k
mu0 = analyzer.mu0
G_mesh = (4 * k**2 * B0_mesh**2) / (3 * mu0**2 * J0_mesh**2 * r_mid**2)

fig6 = plt.figure(figsize=(8, 7))
ax6 = plt.subplot(111)

levels = np.linspace(-2, 4, 25)
contour = ax6.contourf(B0_mesh, J0_mesh/1e6, np.log10(G_mesh), levels=levels,
                       cmap='RdYlBu_r', extend='both')
cbar = plt.colorbar(contour, ax=ax6)
cbar.set_label('log₁₀(G)')

contour_lines = ax6.contour(B0_mesh, J0_mesh/1e6, G_mesh,
                            levels=[0.1, 0.3, 1, 3, 10, 30, 100],
                            colors='k', linewidths=1.5, linestyles='--')
ax6.clabel(contour_lines, inline=True, fontsize=9, fmt='%d')

ax6.contour(B0_mesh, J0_mesh/1e6, G_mesh, levels=[critical_G],
            colors='red', linewidths=3, linestyles='-')

ax6.text(0.15, 6, 'UNSTABLE\n(G < 1)', ha='center', va='center',
         fontsize=12, color='red', fontweight='bold',
         bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
ax6.text(3, 1, 'STABLE\n(G > 1)', ha='center', va='center',
         fontsize=12, color='blue', fontweight='bold',
         bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

ax6.plot(analyzer.B0, analyzer.J0/1e6, 'ko', markersize=10,
         markerfacecolor='yellow', markeredgewidth=2,
         label='Reference')

ax6.set_xlabel('Axial Magnetic Field B₀ (T)')
ax6.set_ylabel('Current Density J₀ (MA/m²)')
ax6.set_title('Figure 6: Design Space - Geometric Stability Index G')
ax6.set_xscale('log')
ax6.set_yscale('log')
ax6.set_xlim([0.1, 10])
ax6.set_ylim([0.1, 10])
ax6.legend(loc='lower left')
ax6.grid(True, alpha=0.3, which='both')

plt.tight_layout()
plt.savefig('Fig6_design_space_G_corrected.png', dpi=300, bbox_inches='tight')
plt.close(fig6)
print("  ✓ Fig6_design_space_G_corrected.png saved")

#=============================================================================
# FIGURE 7: Exhaust Velocity with Stability Contours
#=============================================================================

print("Generating Figure 7: Exhaust Velocity with Stability Contours...")

rho = analyzer.rho
ve_mesh = J0_mesh / np.sqrt(rho * mu0) / 1000

fig7 = plt.figure(figsize=(8, 7))
ax7 = plt.subplot(111)

contour_ve = ax7.contourf(B0_mesh, J0_mesh/1e6, ve_mesh, levels=25,
                          cmap='plasma', extend='max')
cbar_ve = plt.colorbar(contour_ve, ax=ax7)
cbar_ve.set_label('Exhaust Velocity v_e (km/s)')

# Combine and sort levels to ensure they are increasing
G_levels_base = [0.3, 1, 3, 10, 30, 100]
G_levels_all = sorted(list(set(G_levels_base + [critical_G])))

G_contours = ax7.contour(B0_mesh, J0_mesh/1e6, G_mesh,
                         levels=G_levels_all,
                         colors='white', linewidths=2, linestyles='-')
ax7.clabel(G_contours, inline=True, fontsize=9, fmt='G=%d', colors='white')

ax7.plot(analyzer.B0, analyzer.J0/1e6, 'ko', markersize=10,
         markerfacecolor='yellow', markeredgecolor='black', markeredgewidth=2,
         label='Reference')

ax7.annotate('↑ Performance', xy=(1, 3), xytext=(0.3, 6),
            arrowprops=dict(arrowstyle='->', color='red', linewidth=2),
            fontsize=11, color='red', fontweight='bold')
ax7.annotate('↑ Stability', xy=(3, 1), xytext=(6, 0.3),
            arrowprops=dict(arrowstyle='->', color='blue', linewidth=2),
            fontsize=11, color='blue', fontweight='bold')

ax7.set_xlabel('Axial Magnetic Field B₀ (T)')
ax7.set_ylabel('Current Density J₀ (MA/m²)')
ax7.set_title('Figure 7: Exhaust Velocity with Stability Contours')
ax7.set_xscale('log')
ax7.set_yscale('log')
ax7.set_xlim([0.1, 10])
ax7.set_ylim([0.1, 10])
ax7.legend(loc='lower left')
ax7.grid(True, alpha=0.3, which='both')

plt.tight_layout()
plt.savefig('Fig7_exhaust_velocity_corrected.png', dpi=300, bbox_inches='tight')
plt.close(fig7)
print("  ✓ Fig7_exhaust_velocity_corrected.png saved")

#=============================================================================
# FIGURE 8: Parameter Summary
#=============================================================================

print("Generating Figure 8: Parameter Summary...")

fig8 = plt.figure(figsize=(8, 6))
ax8 = plt.subplot(111)
ax8.axis('off')

table_data = [
    ['Parameter', 'Symbol', 'Value', 'Unit'],
    ['Thruster radius', 'R', f'{analyzer.R}', 'm'],
    ['Axial field (ref)', 'B₀', f'{analyzer.B0}', 'T'],
    ['Current density', 'J₀', f'{analyzer.J0:.1e}', 'A/m²'],
    ['Wavenumber', 'k', f'{analyzer.k}', 'm⁻¹'],
    ['Plasma density', 'ρ', f'{analyzer.rho}', 'kg/m³'],
    ['', '', '', ''],
    ['CRITICAL VALUES', '', '', ''],
    ['Critical axial field', 'B₀_crit', f'{critical_B0:.3f}', 'T'],
    ['Critical G index', 'G_crit', f'{critical_G:.2f}', ''],
    ['Critical scale', 'B₀/B₀₀', f'{critical_scale:.2f}', ''],
    ['Min eigenvalue', 'λ_min', f'{eigenvalues_full[0]:.2e}', ''],
    ['', '', '', ''],
    ['DERIVED', '', '', ''],
    ['C = μ₀J₀/2', 'C', f'{analyzer.C:.2f}', 'T/m'],
    ['G at r=R/2', 'G_mid', f'{analyzer.G_mid:.1f}', ''],
    ['Alfvén speed', 'c_A', f'{np.sqrt(analyzer.B0**2/(analyzer.mu0*analyzer.rho))/1000:.1f}', 'km/s'],
]

table = ax8.table(cellText=table_data, loc='center', cellLoc='left', colWidths=[0.2, 0.15, 0.25, 0.15])
table.auto_set_font_size(False)
table.set_fontsize(11)
table.scale(1, 1.8)

for i, row in enumerate(table_data):
    for j in range(4):
        cell = table[(i, j)]
        if i == 0:
            cell.set_facecolor('#40466e')
            cell.set_text_props(weight='bold', color='white')
        elif i == 6 or i == 12:
            cell.set_facecolor('#e6e6e6')
            cell.set_text_props(weight='bold')
        elif i > 6 and i < 12 and j == 2:
            cell.set_text_props(weight='bold', color='red')

ax8.set_title('Figure 8: Summary of Critical Parameters (Full Operator)', fontsize=14, y=0.95)

plt.tight_layout()
plt.savefig('Fig8_parameter_summary_corrected.png', dpi=300, bbox_inches='tight')
plt.close(fig8)
print("  ✓ Fig8_parameter_summary_corrected.png saved")

#=============================================================================
# COMPARISON WITH CLASSICAL KRUSKAL-SHAFRANOV LIMIT
#=============================================================================

print("\n" + "="*80)
print("COMPARISON WITH CLASSICAL KRUSKAL-SHAFRANOV LIMIT")
print("="*80)

# Safety factor q for reference configuration
q_ref = (2 * analyzer.B0) / (analyzer.mu0 * analyzer.J0 * analyzer.R)
print(f"\nSafety factor q (reference): {q_ref:.3f}")

# Relationship between G and q
kR = analyzer.k * analyzer.R
G_from_q = (4 * kR**2 / 3) * q_ref**2
print(f"G predicted from q: {G_from_q:.1f}")
print(f"G computed directly: {analyzer.G_mid:.1f}")

# Critical safety factor from G_crit
q_crit = np.sqrt(3 * critical_G / (4 * kR**2))
print(f"\nCritical safety factor (from G_crit): {q_crit:.3f}")
print(f"Classical Kruskal-Shafranov limit: q > 1")

#=============================================================================
# SUMMARY
#=============================================================================

print("\n" + "="*80)
print("FIGURE GENERATION COMPLETE - CORRECTED VERSION")
print("="*80)
print("\nThe following corrected figures have been created:")
print("  1. Fig1_equilibrium_potential_corrected.png - Full operator coefficients")
print("  2. Fig2_geometric_index_corrected.png - Radial profile of G(r)")
print("  3. Fig3_eigenvalue_spectrum_corrected.png - Full Sturm-Liouville spectrum")
print("  4. Fig4_stability_threshold_corrected.png - Stability vs B₀")
print("  5. Fig5_G_lambda_correlation_corrected.png - G vs λ_min correlation")
print("  6. Fig6_design_space_G_corrected.png - Design space with G contours")
print("  7. Fig7_exhaust_velocity_corrected.png - Exhaust velocity")
print("  8. Fig8_parameter_summary_corrected.png - Summary of critical parameters")
print("\nAll figures saved at 300 DPI for publication.")
print("="*80)

CORRECTED MHD STABILITY ANALYSIS - FULL STURM-LIOUVILLE PROBLEM

Initializing MPD thruster configuration with FULL RADIAL OPERATOR:
  Radius R = 0.1 m
  Axial field B₀ = 0.5 T
  Current density J₀ = 5.0e+06 A/m²
  Axial wavenumber k = 20 m⁻¹
  Plasma density ρ = 1e-05 kg/m³
  Geometric index at r=R/2: G = 1350.9

Solving full Sturm-Liouville problem...
  Minimum eigenvalue: λ_min = -1.7487e+14
  First 5 eigenvalues: [-1.74874635e+14 -9.21407546e+13 -5.18937116e+13  3.23849355e+13
  1.10189273e+14]

Performing stability scan over B₀...



Critical parameters:
  B₀_crit = 0.050 T
  G_crit = 13.51
  Scale factor = 0.10

Generating Figure 1: Equilibrium Configuration...


  ✓ Fig1_equilibrium_potential_corrected.png saved
Generating Figure 2: Geometric Stability Index...


  ✓ Fig2_geometric_index_corrected.png saved
Generating Figure 3: Hessian Eigenvalue Spectrum (Full Operator)...


  ✓ Fig3_eigenvalue_spectrum_corrected.png saved
Generating Figure 4: Stability Threshold Analysis...


  ✓ Fig4_stability_threshold_corrected.png saved
Generating Figure 5: G vs λ_min Correlation...


  ✓ Fig5_G_lambda_correlation_corrected.png saved
Generating Figure 6: Design Space - G Contours...


  ✓ Fig6_design_space_G_corrected.png saved
Generating Figure 7: Exhaust Velocity with Stability Contours...


  ✓ Fig7_exhaust_velocity_corrected.png saved
Generating Figure 8: Parameter Summary...


  ✓ Fig8_parameter_summary_corrected.png saved

COMPARISON WITH CLASSICAL KRUSKAL-SHAFRANOV LIMIT

Safety factor q (reference): 1.592
G predicted from q: 13.5
G computed directly: 1350.9

Critical safety factor (from G_crit): 1.592
Classical Kruskal-Shafranov limit: q > 1

FIGURE GENERATION COMPLETE - CORRECTED VERSION

The following corrected figures have been created:
  1. Fig1_equilibrium_potential_corrected.png - Full operator coefficients
  2. Fig2_geometric_index_corrected.png - Radial profile of G(r)
  3. Fig3_eigenvalue_spectrum_corrected.png - Full Sturm-Liouville spectrum
  4. Fig4_stability_threshold_corrected.png - Stability vs B₀
  5. Fig5_G_lambda_correlation_corrected.png - G vs λ_min correlation
  6. Fig6_design_space_G_corrected.png - Design space with G contours
  7. Fig7_exhaust_velocity_corrected.png - Exhaust velocity
  8. Fig8_parameter_summary_corrected.png - Summary of critical parameters

All figures saved at 300 DPI for publication.
